# Finetune Gemma 3n With Unsloth

Adapted from tutorial found here: https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Gemma3N_(4B)-Conversational.ipynb

### Installation

In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    ! pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    ! pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    ! pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    ! pip install --no-deps unsloth
! pip install transformers==4.55.4
! pip install --no-deps trl==0.22.2
! pip install evaluate
! pip install sacrebleu
! pip install weave
import torch; torch._dynamo.config.recompile_limit = 64;
import weave


In [ ]:
%%capture
!pip install --no-deps --upgrade timm # Only for Gemma 3N

### Load Model

In [ ]:
from unsloth import FastModel
import torch

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3n-E2B-it",
    dtype = None, # None for auto detection
    max_seq_length = 1024, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

# Add LoRA

We now add LoRA adapters so we only need to update a small amount of parameters!

In [ ]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # Should leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

<a name="Data"></a>
### Data Prep
We now use the `Gemma-3` format for conversation style finetunes. Gemma-3 renders multi turn conversations like below:

```
<bos><start_of_turn>user
Hello!<end_of_turn>
<start_of_turn>model
Hey there!<end_of_turn>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3, phi4, qwen2.5, gemma3` and more.

In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

### Load Data

In [ ]:
from datasets import load_dataset
from unsloth.chat_templates import standardize_data_formats

dataset = load_dataset("billingsmoore/LotsawaHouse-bo-en", split = "train[:300]").train_test_split(test_size = 0.01)
dataset['train'] = standardize_data_formats(dataset['train'])
dataset['test'] = standardize_data_formats(dataset['test'])

We now use `standardize_data_formats` to try converting datasets to the correct format for finetuning purposes!

We now have to apply the chat template for `Gemma-3` onto the conversations, and save it to `text`. We remove the `<bos>` token using removeprefix(`'<bos>'`) since we're finetuning. The Processor will add this token before training and the model expects only one.

### Format Data

In [ ]:
def formatting_prompts_func(examples):
   print(examples)
   texts = [f"""<start_of_turn>user\nTranslate Tibetan to English: {bo}<end_of_turn\n<start_of_turn>model\n{en}""" for bo, en in zip(examples['bo'], examples['en'])]
   return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)

### Train the model

In [ ]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    eval_dataset = dataset['test'],
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        eval_steps = 20,
        eval_strategy='steps',
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "wandb", # Use this for WandB etc
    ),
)

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes!

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

# Train

To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [ ]:
trainer_stats = trainer.train()

# Evaluate

In [ ]:
from unsloth.chat_templates import get_chat_template
import re

def translate(input, max_new_tokens = 128, tokenizer=tokenizer):
  tokenizer = get_chat_template(
      tokenizer,
      chat_template = "gemma-3",
  )

  messages = [{
      "role": "user",
      "content": [{
          "type" : "text",
          "text" : f"Translate Tibetan to English: {input['bo']}",
      }]
  }]
  inputs = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt = True, # Must add for generation
      return_tensors = "pt",
      tokenize = True,
      return_dict = True,
  ).to("cuda")
  outputs = model.generate(
      **inputs,
      max_new_tokens = max_new_tokens, # Increase for longer outputs!
      # Recommended Gemma-3 settings!
      temperature = 1.0, top_p = 0.95, top_k = 64,
      # streamer = TextStreamer(tokenizer, skip_prompt = True),

  )

  out = tokenizer.batch_decode(outputs)

  # Modified regex to remove the semicolon and handle no match gracefully
  match = re.search(r"<start_of_turn>model\n(.*?)<end_of_turn>", out[0], re.DOTALL)
  translation = match.group(1).strip().replace(';', '') if match else "" # Return empty string if no match

  return translation

In [ ]:
import evaluate

bleu = evaluate.load("sacrebleu")
chrf = evaluate.load("chrf")
ter  = evaluate.load("ter")

import torch

all_preds = [translate(elt) for elt in dataset['test'].select(range(2))]
all_labels = [elt['en'] for elt in dataset['test'].select(range(2))]




In [ ]:
# Compute metrics
bleu_score  = bleu.compute(predictions=all_preds, references=all_labels)["score"]
chrf_score  = chrf.compute(predictions=all_preds, references=all_labels)["score"]
ter_score   = ter.compute(predictions=all_preds, references=all_labels)["score"]

print({
    "bleu": round(bleu_score, 4),
    "chrf": round(chrf_score, 4),
    "ter" : round(ter_score, 4),
})

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Gemma-3` team, the recommended settings for inference are `temperature = 1.0, top_p = 0.95, top_k = 64`

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("gemma-3n")  # Local saving
tokenizer.save_pretrained("gemma-3n")
# model.push_to_hub("HF_ACCOUNT/gemma-3n", token = "...") # Online saving
# tokenizer.push_to_hub("HF_ACCOUNT/gemma-3n", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastModel
    model, tokenizer = FastModel.from_pretrained(
        model_name = "gemma-3n", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
    )

messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "What is Gemma-3N?",}]
}]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
    tokenize = True,
    return_dict = True,
).to("cuda")

from transformers import TextStreamer
_ = model.generate(
    **inputs,
    max_new_tokens = 128, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

### Saving to float16 for VLLM

We also support saving to `float16` directly for deployment! We save it in the folder `gemma-3N-finetune`. Set `if False` to `if True` to let it run!

In [ ]:
if False: # Change to True to save finetune!
    model.save_pretrained_merged("gemma-3N-finetune", tokenizer)

If you want to upload / push to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload finetune
    model.push_to_hub_merged(
        "HF_ACCOUNT/gemma-3N-finetune", tokenizer,
        token = "hf_..."
    )